# Building a function to calculate the sensitivity

## Recap

In [1]:
import torch

def get_parallel_db(db, remove_index):
    return(torch.cat((db[:remove_index], db[remove_index+1:])))

def get_parallel_dbs(db):

    pdbs = []

    for i in range(len(db)):
        pdbs.append(get_parallel_db(db, i))

    return(pdbs)

def create_db_and_parallel_dbs(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return(db, pdbs)

## Building the actual function

In [2]:
def sensitivity(query, num_entries=1000):

    db, pdbs = create_db_and_parallel_dbs(num_entries)

    full_db_result = query(db)

    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance>max_distance):
            max_distance = db_distance

    return(max_distance)

In [3]:
# Calculate the sensitivity of sum function(Testing)

def sumQuery(db):
    return(db.sum())

print(sensitivity(sumQuery))

tensor(1)


## Let's check out the sensitivity for the mean query

In [4]:
def query(db):
    return(db.float().mean())

In [5]:
print(sensitivity(query))

tensor(0.0005)


### Inference

The sensitivity of the query decreases by a lot because we are dividing it by the number of entries in the database. This corresponds to the average value in the database. (0.5/1000)